## RNNs on Larger Dataset

In [1]:
# imports
# packages used
import librosa
import os 

import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.notebook import tqdm, trange
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

## Prepare Data

### Transform all audio files using Librosa
audio data are under mtg-jamendo-dataset/dataset 


github repository:  
https://github.com/MTG/mtg-jamendo-dataset/tree/master?tab=readme-ov-file#downloading-the-data


reference:   
Bogdanov, D., Won M., Tovstogan P., Porter A., & Serra X. (2019). The MTG-Jamendo Dataset for Automatic Music Tagging. Machine Learning for Music Discovery Workshop, International Conference on Machine Learning (ICML 2019).


citation:   
@conference {bogdanov2019mtg,
    author = "Bogdanov, Dmitry and Won, Minz and Tovstogan, Philip and Porter, Alastair and Serra, Xavier",
    title = "The MTG-Jamendo Dataset for Automatic Music Tagging",
    booktitle = "Machine Learning for Music Discovery Workshop, International Conference on Machine Learning (ICML 2019)",
    year = "2019",
    address = "Long Beach, CA, United States",
    url = "http://hdl.handle.net/10230/42015"
}

In [5]:
# Define a function to extract features from a single audio file
def extract_features(audio_path, file_id):
    y, sr = librosa.load(audio_path, sr=None)
    features = {
        'file_id': file_id,
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr),
        'chroma_cqt': librosa.feature.chroma_cqt(y=y, sr=sr),
        'chroma_vqt': librosa.feature.chroma_vqt(y=y, sr=sr, intervals='ji5'),
        'melspectrogram': librosa.feature.melspectrogram(y=y, sr=sr),
        'mfcc': librosa.feature.mfcc(y=y, sr=sr),
        'rms': librosa.feature.rms(y=y),
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr),
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr),
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr),
        'spectral_flatness': librosa.feature.spectral_flatness(y=y),
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr),
        'tonnetz': librosa.feature.tonnetz(y=y, sr=sr),
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(y=y),
        'tempo': librosa.feature.tempo(y=y, sr=sr),
    }
    return features

# Define a path to the dataset folder and the new output folder
dataset_path = 'C:/Users/Xiyah/OneDrive - Johns Hopkins/JHU/2024Spring/EN.553.602.01DataMining/Project/mtg-jamendo-dataset/dataset'
output_path = 'mtg-jamendo-dataset/extracted_features'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Process each folder and file, starting from '05'
start_folder = '25'  # Define the folder from which to start processing
start_processing = False

# Process each folder and file
for folder in sorted(os.listdir(dataset_path)):
    if folder >= start_folder:
        start_processing = True
    if not start_processing:
        continue

    folder_path = os.path.join(dataset_path, folder)
    all_features = []  # List to hold all features for this folder

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        for file in os.listdir(folder_path):
            if file.endswith('.mp3'):
                file_path = os.path.join(folder_path, file)
                # Extract the numerical part of the filename
                file_id = file.split('.')[0].split('_')[0]
                try:
                    features = extract_features(file_path, file_id)
                    all_features.append(features)
                    print(f"Processed file: {file}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

        # Convert feature list to numpy array and save as .npy file
        feature_array = np.array(all_features)
        np.save(os.path.join(output_path, f'features_{folder}.npy'), feature_array)
        print(f"Features saved to '{output_path}/features_{folder}.npy'")

print("Feature extraction complete for all folders.")

Processing folder: 05
Processed file: 1012005.low.mp3
Processed file: 1023305.low.mp3
Processed file: 1024405.low.mp3
Processed file: 1027705.low.mp3
Processed file: 1028505.low.mp3
Processed file: 1028905.low.mp3
Processed file: 1031105.low.mp3
Processed file: 1036105.low.mp3
Processed file: 1048305.low.mp3
Processed file: 1049705.low.mp3
Processed file: 1053505.low.mp3
Processed file: 1056405.low.mp3
Processed file: 1061405.low.mp3
Processed file: 1062505.low.mp3
Processed file: 1066205.low.mp3
Processed file: 1066505.low.mp3
Processed file: 1066805.low.mp3
Processed file: 1068405.low.mp3
Processed file: 1070605.low.mp3
Processed file: 1072505.low.mp3
Processed file: 1073705.low.mp3
Processed file: 1078705.low.mp3
Processed file: 1079805.low.mp3
Processed file: 1081705.low.mp3
Processed file: 1082105.low.mp3
Processed file: 1083305.low.mp3
Processed file: 108505.low.mp3
Processed file: 1087105.low.mp3
Processed file: 1095105.low.mp3
Processed file: 1108005.low.mp3
Processed file: 110

KeyboardInterrupt: 